# Tree of Thought approach


#### Paper 
https://arxiv.org/pdf/2305.10601.pdf

This notebook demonstrates the use of Tree of thought as a 4 step process for a creative writing task.

1. Create *K* plans to approach the task
2. Vote the best 1 out of the *K* plans
3. Create *N* passages with the best plan
4. Vote the best 1 out of the *N* passages

#### Task

In this demonstration, we will use LLM to create a product marketing slogan. The marketing lead has some thoughts in mind that she would like the LLM to follow. Refer to the standard prompt that describes the marketing lead's preferences.


#### Note:
1. Since we are using commerical API, it will cost you $ to run the code
2. The code can be adjusted for similar tasks by tweaking the prompts, rest of the code stays the same

#### Langchain
Use of Langchain will simplify the code below. Langchain has **NOT** been used deliberately to show you the concept of LLM chains :) in the next iteration you will use Langchain to rewrite this code. If you already know Langchain, then feel free to use it instead of the solution proposed in this notebook.

#### Google Colab
If you are running the code in Google colab, install the packages by uncommenting/running the cell below

* The API key file file will not be available
* You will be prompted to provide the Cohere API Token

Uncomment & run the code in the cell below:

In [20]:
## The script is downloaded and run to setup the utils folder

# !curl -H "Accept: application/vnd.github.VERSION.raw" https://raw.githubusercontent.com/acloudfan/gen-ai-app-dev/main/Setup/gcsetup.sh  > gcsetup.sh
# !chmod u+x gcsetup.sh
# !./gcsetup.sh -l

## Setup the environment variable for API access

#### NOTE
You MUST change the path to your own environment file

In [1]:
from dotenv import load_dotenv
import os
import sys

import warnings

warnings.filterwarnings("ignore")

# Load the file that contains the API keys
load_dotenv('C:\\Users\\raj\\.jupyter\\.env')

True

In [ ]:
# Setting path so we can access the utils folder
sys.path.append('../')
sys.path.append('./')

from utils.api_key_check_utility import api_key_check

# Check if the Open AI key is available
cohere_api_key = api_key_check("OPENAI_API_KEY")

### Use OpenAI to run infrence

https://platform.openai.com/docs/api-reference/chat

In [2]:
from openai import OpenAI

client = OpenAI()

# utility function for running infrence
def  run_openai_infrence(prompt, temperature=0.5, max_tokens=400, stop_sequence=[], count=1):
    client = OpenAI()
    
    response_content = []

    for i in range(count):
        response = client.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=[
            {
                "role": "system", 
                "content": "you are an expert in generating product marketing material"
            },
            {
              "role": "user",
              "content": prompt
            }
          ],
          temperature=temperature,
          max_tokens=max_tokens,
          stop=stop_sequence
        )

        response_content.append(response.choices[0].message.content)
        
    return response_content

## Response without ToT

We will first create a prompt and get a response from Open AI/GPT.

In [3]:
# These are output indicators  used in the prompt
ouput_passage_indicator = "Passage:"
ouput_plan_indicator = "Plan:"
output_best_plan_indicator = "Best Plan:"
output_best_passage_indicator = "Best Passage:"

In [4]:
# This is a standard prompt
prompt_instructions = """Write a passage to describe our new product:
Javelin 20 High Performance Shoes.

The passage should be no more than 100 words.

The title must contain the sentence: 
A new you

The first sentence must contain a few of these words: 
motivation, workout, sports, attractive

The last few sentences must motivate the reader to visit out website http://acme.com/jav20

The passage must contain the phrase : 
trend setter
"""

prompt = prompt_instructions + ouput_passage_indicator + "\nYour passage here."
# print( prompt )

In [5]:
# response to the standard prompt
response = run_openai_infrence(prompt)

print(response[0])

Title: A new you with Javelin 20 High Performance Shoes

Looking for motivation to enhance your workout or sports performance? The Javelin 20 High Performance Shoes are designed to not only boost your athletic abilities but also make a stylish statement. These attractive shoes are the perfect blend of comfort and functionality, making you a trend setter in the world of sports footwear. Elevate your game and take the first step towards a new you with Javelin 20. Visit our website at http://acme.com/jav20 to discover the ultimate in athletic footwear.


## Response with ToT

### 1. Generate 3 plans

In [6]:
prompt = prompt_instructions + """
Your output will be in following format.

{ouput_plan_indicator}
Your plan here.

{ouput_passage_indicator}
Your passage here.
""" 

prompt = prompt.format(ouput_plan_indicator=ouput_plan_indicator, ouput_passage_indicator=ouput_passage_indicator)


print(prompt)

Write a passage to describe our new product:
Javelin 20 High Performance Shoes.

The passage should be no more than 100 words.

The title must contain the sentence: 
A new you

The first sentence must contain a few of these words: 
motivation, workout, sports, attractive

The last few sentences must motivate the reader to visit out website http://acme.com/jav20

The passage must contain the phrase : 
trend setter

Your output will be in following format.

Plan:
Your plan here.

Passage:
Your passage here.



In [7]:
# use stop sequence to generate just the plan
passage_stop_sequence = [ouput_passage_indicator]

# This will return an array with 3 elements
plans_for_passage = run_openai_infrence(prompt,0.8,400,passage_stop_sequence,3)

In [8]:
for plan in plans_for_passage:
    print(plan)

Plan:
Highlight the motivational aspect of the Javelin 20 High Performance Shoes, emphasizing their appeal for workouts, sports, and an active lifestyle. Include a call-to-action to visit the website for more information.


Plan:
Highlight the motivation and attractiveness of the Javelin 20 High Performance Shoes, mentioning their suitability for both workouts and sports. End with an enticing call-to-action to visit the website.


Plan:
Incorporate words like motivation, workout, sports, and attractive to emphasize the benefits of the Javelin 20 High Performance Shoes. Conclude with a strong call-to-action encouraging readers to visit the website.




### 2. Ask LLM to vote on the plan

In [9]:
prompt_plan_vote = """
3 experts were given the following task.

Task:

{prompt_instructions}

you need to review the 3 plans and identify the best plan out of the 3. 

Output your response as follows, where N is the plan number. 
{output_best_plan_indicator} N

Give 3 reasons why you select the plan.

{plans}

"""

prompt_plans = ""
for i, plan in enumerate(plans_for_passage):
    plan = plan.replace(ouput_plan_indicator, '')
    prompt_plan = "Plan {i}:{plan}".format(i=(i+1), plan=plan)
    
    prompt_plans = prompt_plans + prompt_plan

prompt = prompt_plan_vote.format(prompt_instructions=prompt_instructions, plans = prompt_plans, output_best_plan_indicator=output_best_plan_indicator)
print(prompt)


3 experts were given the following task.

Task:

Write a passage to describe our new product:
Javelin 20 High Performance Shoes.

The passage should be no more than 100 words.

The title must contain the sentence: 
A new you

The first sentence must contain a few of these words: 
motivation, workout, sports, attractive

The last few sentences must motivate the reader to visit out website http://acme.com/jav20

The passage must contain the phrase : 
trend setter


you need to review the 3 plans and identify the best plan out of the 3. 

Output your response as follows, where N is the plan number. 
Best Plan: N

Give 3 reasons why you select the plan.

Plan 1:
Highlight the motivational aspect of the Javelin 20 High Performance Shoes, emphasizing their appeal for workouts, sports, and an active lifestyle. Include a call-to-action to visit the website for more information.

Plan 2:
Highlight the motivation and attractiveness of the Javelin 20 High Performance Shoes, mentioning their suit

In [10]:
# use stop sequence to generate just the plan
passage_stop_sequence = [ouput_passage_indicator]

# This will return an array with 3 elements
plan_voted_best = run_openai_infrence(prompt,0.8,400)

In [11]:
# Get the index of the best plan by searching for the "Best Plan:"
best_plan_index = plan_voted_best[0].find(output_best_plan_indicator)
temp_index = best_plan_index+len(output_best_plan_indicator)
new_line_index  = plan_voted_best[0].find("\n",temp_index)
best_plan_index = int(plan_voted_best[0][temp_index : new_line_index]) - 1

print('Best Plan Index:', best_plan_index)
print('------')
print(plan_voted_best[0])

Best Plan Index: 1
------
Best Plan: 2

Three reasons for selecting Plan 2:

1. Plan 2 effectively combines the elements of motivation and attractiveness to appeal to a wider audience, enhancing the overall desirability of the Javelin 20 High Performance Shoes.
   
2. By highlighting the suitability of the shoes for both workouts and sports, Plan 2 showcases the versatility of the product, making it more appealing to customers with different fitness interests.
   
3. The enticing call-to-action at the end of Plan 2 creates a sense of urgency and encourages immediate engagement, driving traffic to the website effectively.




## 3. Prompt LLM to generate 3 passages using the best plan

In [12]:
prompt_generate_3_passages = """
Create three passages using the following instructions and plan.

Instructions:
{prompt_instructions}

{best_plan}

Your output will be in following format.

{ouput_passage_indicator}
Your response here


"""

# Use the plans generated in step-1, best plan index from step 2
best_plan = plans_for_passage[best_plan_index]

prompt = prompt_generate_3_passages.format(prompt_instructions=prompt_instructions, 
                                           best_plan=best_plan,
                                           ouput_passage_indicator=ouput_passage_indicator)


In [13]:
print(prompt)


Create three passages using the following instructions and plan.

Instructions:
Write a passage to describe our new product:
Javelin 20 High Performance Shoes.

The passage should be no more than 100 words.

The title must contain the sentence: 
A new you

The first sentence must contain a few of these words: 
motivation, workout, sports, attractive

The last few sentences must motivate the reader to visit out website http://acme.com/jav20

The passage must contain the phrase : 
trend setter


Plan:
Highlight the motivation and attractiveness of the Javelin 20 High Performance Shoes, mentioning their suitability for both workouts and sports. End with an enticing call-to-action to visit the website.



Your output will be in following format.

Passage:
Your response here





In [14]:
# This will return an array with 3 elements
passages_based_on_best_plan = run_openai_infrence(prompt,temperature=0.5,max_tokens=400,stop_sequence=[],count=3)

In [15]:
for passage in passages_based_on_best_plan:
    print(passage)

Passage:
A new you awaits with the Javelin 20 High Performance Shoes, designed to boost your motivation during workouts and sports activities. These attractive shoes are not just stylish but also functional, offering top-notch performance and comfort. Be a trend setter in your fitness journey with the Javelin 20. Elevate your game and reach your peak potential with every step you take in these high-performance shoes. Ready to experience the difference? Visit our website at http://acme.com/jav20 and unleash your full potential today.
Passage:
A new you awaits with the Javelin 20 High Performance Shoes, designed to elevate your motivation and style during workouts and sports. These attractive shoes are a trend setter in performance footwear, offering unparalleled support and comfort. Step up your game and experience the difference with the Javelin 20. Don't miss out on this opportunity to enhance your performance - visit our website at http://acme.com/jav20 now and be the first to own a 

### 4. Vote the best out of the 3 passages

In [16]:
prompt_passage_vote = """
3 experts were given the following task.

Task:
{prompt_instructions}

you need to review the 3 passages and identify the best passage out of the 3. 

Output your response as follows, where N is the passage number. 
{output_best_passage_indicator} N

Give 3 reasons why you select the passage.

{passages}

"""

prompt_passages = ""
for i, passage in enumerate(passages_based_on_best_plan):
    passage = passage.replace(ouput_passage_indicator, '')
    prompt_passage = "Passage {i}:{passage}\n\n".format(i=(i+1), passage=passage)
    
    prompt_passages = prompt_passages + prompt_passage

prompt = prompt_passage_vote.format(prompt_instructions=prompt_instructions, 
                                    passages = prompt_passages, 
                                    output_best_passage_indicator=output_best_passage_indicator)
print(prompt)


3 experts were given the following task.

Task:
Write a passage to describe our new product:
Javelin 20 High Performance Shoes.

The passage should be no more than 100 words.

The title must contain the sentence: 
A new you

The first sentence must contain a few of these words: 
motivation, workout, sports, attractive

The last few sentences must motivate the reader to visit out website http://acme.com/jav20

The passage must contain the phrase : 
trend setter


you need to review the 3 passages and identify the best passage out of the 3. 

Output your response as follows, where N is the passage number. 
Best Passage: N

Give 3 reasons why you select the passage.

Passage 1:
A new you awaits with the Javelin 20 High Performance Shoes, designed to boost your motivation during workouts and sports activities. These attractive shoes are not just stylish but also functional, offering top-notch performance and comfort. Be a trend setter in your fitness journey with the Javelin 20. Elevate y

In [17]:
# use stop sequence to generate just the plan
passage_stop_sequence = [ouput_passage_indicator]

# This will return an array with 3 elements
passage_voted_best = run_openai_infrence(prompt,0.8,400)

In [18]:
# Get the index of the best pasage by searching for the "Best Passage:"
best_passage_index = passage_voted_best[0].find(output_best_passage_indicator)
temp_index = best_passage_index+len(output_best_passage_indicator)
new_line_index  = passage_voted_best[0].find("\n",temp_index)
best_passage_index = int(passage_voted_best[0][temp_index : new_line_index]) - 1

print('Best Passage Index:', best_passage_index)
print('------')
print(passage_voted_best[0])

Best Passage Index: 1
------
Best Passage: 2

Reasons for Selection:
1. Use of the phrase "elevate your motivation and style" effectively captures the essence of the product, appealing to both performance and aesthetics.
2. The mention of being a trend setter in performance footwear positions the Javelin 20 as a cutting-edge choice in the market.
3. The call-to-action at the end is compelling, urging the reader to visit the website immediately to explore the product further and make a purchase.


In [19]:
print("Final Result")
print("------------")
print(passages_based_on_best_plan[best_passage_index])

Final Result
------------
Passage:
A new you awaits with the Javelin 20 High Performance Shoes, designed to elevate your motivation and style during workouts and sports. These attractive shoes are a trend setter in performance footwear, offering unparalleled support and comfort. Step up your game and experience the difference with the Javelin 20. Don't miss out on this opportunity to enhance your performance - visit our website at http://acme.com/jav20 now and be the first to own a pair of these exceptional shoes.
